In [1]:
import os
import pickle
import gc
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import load_pickle, dump_pickle, raw_data_path, feature_data_path

In [2]:
def extract_ctr(data, feature, alias):
    '''统计给定数据的转化率
    
    '''
    query_cnt = data.groupby([feature]).size().reset_index().rename(columns={0: 'query_cnt'})
    conversion_cnt = data[data['is_trade'] == 1].groupby([feature]).size().reset_index().rename(columns={0: 'conversion_cnt'})
    ctr = pd.merge(query_cnt, conversion_cnt, how='left', on=[feature])
    ctr[['conversion_cnt']] = ctr[['conversion_cnt']].fillna(0)
    ctr[alias] = ctr['conversion_cnt'] / ctr['query_cnt']
    return ctr

In [9]:
all_data = load_pickle(feature_data_path+'all_data_all_features.pkl')
extract_ctr(all_data, feature='category_predict_rank', alias='category_predict_rank_ctr')

# all_data[['category_predict_rank']]


,category_predict_rank,query_cnt,conversion_cnt,category_predict_rank_ctr
0,-1,47819,737.0,0.015412
1,0,306209,6101.0,0.019924
2,1,89673,1431.0,0.015958
3,2,35765,531.0,0.014847
4,3,12494,142.0,0.011365
5,4,3402,41.0,0.012052
6,5,910,9.0,0.009890
7,6,187,1.0,0.005348
8,7,18,1.0,0.055556
9,8,5,0.0,0.000000


In [81]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

all_data = load_pickle(raw_data_path+'all_data.pkl')


def add_predict_property(property_set, predict_category_property):
    for category_property in predict_category_property.split(";"):
        if category_property != '-1':
            property_list = category_property.split(":")[1].split(',')
            property_set.update(property_list)


property_set = set()

all_data['item_property_list'].apply(lambda x: property_set.update(x.split(";"))) 
all_data['predict_category_property'].apply(lambda x: add_predict_property(property_set, x)) 

le.fit(list(property_set))

LabelEncoder()

In [82]:
all_data = all_data[:10000]

from sklearn.metrics.pairwise import cosine_similarity

def get_predict_consine_sim(le, item_property_list, predict_category_property):
    item_property_list = le.transform(item_property_list.split(";"))
    predict_property_set = set()
    for category_property in predict_category_property.split(";"):
        if category_property != '-1':
            property_list = category_property.split(":")[1].split(',')
            predict_property_set.update(property_list)
    
    predict_property_list = le.transform(list(predict_property_set))
    
    a = np.zeros((1,len(le.classes_)))
    b = np.zeros((1,len(le.classes_)))
    
    a[0][item_property_list] = 1
    b[0][predict_property_list] = 1
    
    return cosine_similarity(a, b)[0][0]

%time all_data['predict_consine_sim'] = all_data.apply(lambda row: get_predict_consine_sim(le, row['item_property_list'], row['predict_category_property']), axis=1)

# all_data['predict_consine_sim'] = all_data.apply(lambda row: get_category_predict_rank(
#         row['item_category_list'], row['predict_category_property']), axis=1)

all_data.head()

CPU times: user 2h 9min 24s, sys: 2min 53s, total: 2h 12min 17s
Wall time: 6min 37s


,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,category2_label,item_property_list0,item_property_list1,item_property_list2,item_property_list3,item_property_list4,item_property_list5,item_property_list6,item_property_list7,predict_consine_sim
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,9,27,66,41,228,259,404,224,87,0.000000
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,9,27,66,41,228,259,404,224,87,0.095346
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,9,27,66,41,228,259,404,224,87,0.213201
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,9,27,66,41,228,259,404,224,87,0.000000
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,9,27,66,41,228,259,404,224,87,0.213201
